# Obesity Classifier


## Setup

- includes
  - determining the computing device
  - model name
  - csv path
  - destinated json line path

In [11]:
import json
import re
from pprint import pprint
 
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    LlamaModel
)
from trl import SFTTrainer
from sklearn.model_selection import train_test_split
 
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-3.2-1B" 
CSV_PATH = "ObesityDataSet.csv"
TRAIN_JSON_PATH = "ObesityTrainDataSet.jsonl"
TEST_JSON_PATH = "ObesityTestDataSet.jsonl"
RANDOM_SEED = 42

## data processing

- convert the csv file to json line file
- split the data into train data and test data, 8:2 ratio

In [12]:

df = pd.read_csv(CSV_PATH)
df = df.rename(columns={"NObeyesdad": "label"})
train_df, test_df = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED)

train_df.to_json(TRAIN_JSON_PATH, orient="records", lines=True)
test_df.to_json(TEST_JSON_PATH, orient="records", lines=True)

## load dataset

- load both test and train datasets into dataset variable

In [13]:
dataset = load_dataset("json", data_files={"train": TRAIN_JSON_PATH, "test": TEST_JSON_PATH})
dataset

Generating train split: 1688 examples [00:00, 137788.47 examples/s]
Generating test split: 423 examples [00:00, 41525.82 examples/s]


DatasetDict({
    train: Dataset({
        features: ['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'label'],
        num_rows: 1688
    })
    test: Dataset({
        features: ['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'label'],
        num_rows: 423
    })
})